In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

In [2]:
# Importação dos conjutos de amostras
dados_treino = pd.read_csv("conjunto_de_treinamento.csv", delimiter=",", decimal=".")
dados_teste = pd.read_csv("conjunto_de_teste.csv", delimiter=",", decimal=".")

In [3]:
dados_teste.head()

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,estado_onde_trabalha,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha
0,20001,1,25,presencial,1,M,37,2,0,0,...,,N,,0,0.0,0.0,0.0,0.0,384.0,384.0
1,20002,1,10,internet,1,F,31,2,0,0,...,RJ,N,,0,9.0,5.0,NaN,NaN,275.0,275.0
2,20003,1,10,internet,1,F,18,2,0,0,...,RS,N,,0,9.0,2.0,NaN,NaN,948.0,948.0
3,20004,1,10,presencial,1,F,55,2,0,0,...,,N,,0,9.0,1.0,0.0,0.0,581.0,581.0
4,20005,1,10,presencial,1,F,55,1,0,0,...,,N,,0,0.0,1.0,0.0,0.0,573.0,573.0


In [4]:
dados_treino.head()

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
0,1,1,10,presencial,1,M,85,2,0,0,...,N,,0,9.0,1.0,0.0,0.0,600.0,600.0,0
1,2,1,25,internet,1,F,38,1,0,0,...,N,,0,2.0,5.0,NaN,NaN,492.0,492.0,0
2,3,1,20,internet,1,F,37,2,0,0,...,N,,0,NaN,NaN,NaN,NaN,450.0,450.0,1
3,4,1,20,internet,1,M,37,1,1,0,...,Y,54,0,9.0,2.0,NaN,NaN,932.0,932.0,1
4,5,7,1,internet,1,F,51,1,3,0,...,N,,0,9.0,5.0,NaN,NaN,440.0,440.0,1


In [5]:
# Verifica perda de dados de treino ao utilizar somente linhas sem dados faltantes
# Testar substituindo os dados faltantes do treino pela média

linha_treino_com_na = dados_treino.shape[0]
linha_treino_sem_na = dados_treino.dropna().shape[0]

print("Perda de %0.2f%% dos dados de treino ao retirar linhas com informações faltantes." %(100 - linha_treino_sem_na*100/linha_treino_com_na))

Perda de 65.12% dos dados de treino ao retirar linhas com informações faltantes.


A perda de 65.12% do dataset de treino pode representar um grande problema pois retirar grande parte da visão de todo o universo das amostrar. Para prosseguir sem perder essas amostras, substitui-se os NaN de forma a manter a característica das distribuições.

Para fazer essa contabilização, primeiro faz-se o tratamento dos dados da base de treino e teste. A seguir seguem as modificaçãoes feitas sobre os dados.

In [6]:
# forma_envio_solicitacao -> Abrir em 3 colunas (internet, correio e presencial)
# sexo -> Abrir em 3 colunas (masculino, feminino, nao informado). Se vazio colocar N
# grau_instrucao -> Remover
# estado_onde_nasceu -> Transformar em regiao com alto indice de inadimplencia
# estado_onde_reside -> Transformar em regiao com alto indice de inadimplencia
# possui_telefone_residencial -> Transformar em 1 - Sim 0 - Nao
# possui_telefone_celular ->  Remover
# qtde_contas_bancarias_especiais -> Remover
# vinculo_formal_com_empresa -> Transformar em 1 - Sim 0 - Nao
# estado_onde_trabalha -> Inútil, apenas 0.2% está preenchido
# possui_telefone_trabalho -> Transformar em 1 - Sim 0 - Nao
# meses_no_trabalho -> Remover

# deleta as colunas desnecessárias/com dados corrompidos
dados_treino = dados_treino.drop(columns=['id_solicitante', 'grau_instrucao', 'possui_telefone_celular',\
                                          'qtde_contas_bancarias_especiais', 'meses_no_trabalho', 'estado_onde_trabalha',\
                                         'codigo_area_telefone_trabalho', 'codigo_area_telefone_residencial'])
dados_teste = dados_teste.drop(columns=['id_solicitante', 'grau_instrucao', 'possui_telefone_celular',\
                                          'qtde_contas_bancarias_especiais', 'meses_no_trabalho', 'estado_onde_trabalha',\
                                         'codigo_area_telefone_trabalho', 'codigo_area_telefone_residencial'])

In [7]:
# splita as colunas sexo e forma_envio_solicitacao em uma coluna para cada opção dentre as possíveis
valores = {'sexo':'N'}

dados_treino = dados_treino.fillna(value=valores)
dados_teste = dados_teste.fillna(value=valores)

dados_treino = pd.get_dummies(dados_treino, columns=['forma_envio_solicitacao', 'sexo'])
dados_teste = pd.get_dummies(dados_teste, columns=['forma_envio_solicitacao', 'sexo'])

dados_treino = dados_treino.drop(columns=['sexo_ '])
dados_teste = dados_teste.drop(columns=['sexo_ '])

In [8]:
# Troca os Y e N por 1 e 0, respectivamente
valores_a_trocar = ['possui_telefone_residencial', 'vinculo_formal_com_empresa', 'possui_telefone_trabalho']

for coluna in valores_a_trocar:
    dados_treino[coluna] = dados_treino[coluna].map({'Y':1, 'N':0}, na_action=None)
    dados_teste[coluna] = dados_teste[coluna].map({'Y':1, 'N':0}, na_action=None)

In [9]:
dados_treino

,produto_solicitado,dia_vencimento,tipo_endereco,idade,estado_civil,qtde_dependentes,nacionalidade,estado_onde_nasceu,estado_onde_reside,possui_telefone_residencial,...,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente,forma_envio_solicitacao_correio,forma_envio_solicitacao_internet,forma_envio_solicitacao_presencial,sexo_F,sexo_M,sexo_N
0,1,10,1,85,2,0,1,CE,CE,1,...,0.0,600.0,600.0,0,0,0,1,0,1,0
1,1,25,1,38,1,0,1,SE,SE,1,...,NaN,492.0,492.0,0,0,1,0,1,0,0
2,1,20,1,37,2,0,1,BA,BA,1,...,NaN,450.0,450.0,1,0,1,0,1,0,0
3,1,20,1,37,1,1,1,RS,RS,1,...,NaN,932.0,932.0,1,0,1,0,0,1,0
4,7,1,1,51,1,3,1,BA,BA,1,...,NaN,440.0,440.0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,10,1,27,2,0,1,MG,MG,1,...,0.0,308.0,308.0,0,0,0,1,0,1,0
19996,1,20,1,26,2,1,1,CE,CE,1,...,0.0,639.0,639.0,0,0,0,1,1,0,0
19997,1,10,1,63,2,0,1,BA,BA,1,...,NaN,486.0,486.0,0,0,1,0,1,0,0
19998,1,5,1,84,1,0,1,PB,RN,0,...,NaN,590.0,590.0,0,0,1,0,1,0,0


In [10]:
import numpy as np

valores_a_trocar = ['estado_onde_nasceu', 'estado_onde_reside']

dicionario = {'AC':0, 'AP':0, 'AM':0, 'PA':0, 'RO':0, 'RR':0, 'TO':0,\
              'MA':1, 'PI':1, 'CE':1, 'RN':1, 'PB':1, 'PE':1, \
             'AL':1, 'SE':1, 'BA':1, 'MT':2, 'MS':2, 'GO':2,\
             'DF':2, 'RJ':3, 'MG':3, 'SP':3, 'ES':3, 'SC':4, 'PR':4, 'RS':4, 'XX':5}

for coluna in valores_a_trocar:
    dados_treino[coluna] = dados_treino[coluna].map(dicionario, na_action=None)
    dados_teste[coluna] = dados_teste[coluna].map(dicionario, na_action=None)

In [11]:
dados_teste = dados_teste.fillna(dados_teste.median())
dados_treino = dados_treino.fillna(dados_treino.median())

In [12]:
dados_treino

,produto_solicitado,dia_vencimento,tipo_endereco,idade,estado_civil,qtde_dependentes,nacionalidade,estado_onde_nasceu,estado_onde_reside,possui_telefone_residencial,...,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente,forma_envio_solicitacao_correio,forma_envio_solicitacao_internet,forma_envio_solicitacao_presencial,sexo_F,sexo_M,sexo_N
0,1,10,1,85,2,0,1,1.0,1,1,...,0.0,600.0,600.0,0,0,0,1,0,1,0
1,1,25,1,38,1,0,1,1.0,1,1,...,0.0,492.0,492.0,0,0,1,0,1,0,0
2,1,20,1,37,2,0,1,1.0,1,1,...,0.0,450.0,450.0,1,0,1,0,1,0,0
3,1,20,1,37,1,1,1,4.0,4,1,...,0.0,932.0,932.0,1,0,1,0,0,1,0
4,7,1,1,51,1,3,1,1.0,1,1,...,0.0,440.0,440.0,1,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,10,1,27,2,0,1,3.0,3,1,...,0.0,308.0,308.0,0,0,0,1,0,1,0
19996,1,20,1,26,2,1,1,1.0,1,1,...,0.0,639.0,639.0,0,0,0,1,1,0,0
19997,1,10,1,63,2,0,1,1.0,1,1,...,0.0,486.0,486.0,0,0,1,0,1,0,0
19998,1,5,1,84,1,0,1,1.0,1,0,...,0.0,590.0,590.0,0,0,1,0,1,0,0


In [13]:
#separacao em alvo e features
# embaralhar antes e testar 150 vezes para cada valor de KNN afim de verificar desvio e performance, tentar corrigir o modelo com regularização

x_treino = dados_treino.iloc[:16000,:-1]
y_treino = dados_treino['inadimplente'].iloc[:16000]

x_teste = dados_treino.iloc[16000:, :-1]
y_teste = dados_treino['inadimplente'].iloc[16000]

In [28]:
print("  K   |  %   ")
for numero_kneighbors in range(2000, 15000, 500):
    classificador = KNeighborsClassifier(n_neighbors=numero_kneighbors, weights='uniform')
    
    classificador = classificador.fit(x_treino, y_treino)
    y_resposta = classificador.predict(x_teste)
    
    total   = 4000
    acertos = sum(y_resposta==y_teste)
    erros   = sum(y_resposta!=y_teste)
    
    acuracia = acertos / total

    print (f"{numero_kneighbors}     %.1f %%" %(100*acuracia))

  K   |  %   
2000     69.8 %
2500     71.3 %
3000     72.3 %
3500     75.7 %
4000     79.8 %
4500     80.0 %
5000     80.2 %
5500     81.1 %
6000     80.7 %
6500     80.7 %
7000     81.0 %
7500     81.5 %
8000     82.0 %
8500     89.0 %
9000     96.9 %
9500     98.3 %
10000     98.7 %
10500     98.9 %
11000     98.7 %
11500     98.7 %
12000     98.9 %
12500     99.1 %
13000     99.8 %
13500     99.8 %
14000     99.9 %
14500     99.3 %
